|        |        |        |
|--------|--------|--------|
![H-BRS](logos/h-brs.png) | ![A2S](logos/a2s.png) | ![b-it](logos/b-it.png) |

# Software Engineering for Robotics

# SER Assignment 1

#### Team members:
* ...
* ...
* ...

# Robot Behaviour Management Using State Machines and Behaviour Trees

In this assignment, we will compare the implementation of behaviour trees and state machines to establish primary safety features for a robot; this includes situations such as the battery level falling below a certain threshold and avoiding potential collisions. In other words, we want to implement both a behaviour tree and a state machine to achieve the same functionality.

We particularly want the robot to behave as follows:
* *The battery level falling below a given threshold*: The robot starts rotating in place until the level is above the threshold again (**you can control the battery level by publishing to a topic you define**).
* *A collision is about to happen*: The robot stops moving and needs to be moved to a safe distance manually.

**You will use the Robile simulation for testing your implementation. For your submission, you need to add your code to the appropriate cells below; however, note that, to actually test your implementation, you need to integrate the code in a ROS package and perform all tests on your local machine.**

## Robot Safety Functionalities Using a State Machine [45 points]

To implement a state machine, we will use the [SMACH framework](https://wiki.ros.org/smach/Tutorials). SMACH as such is ROS independent, but `executive_smach` provides a ROS integration, which we will be using in this assignment.

Your task here is to implement the necessary states and set up the state machine to achieve the desired robot functionality.

In [ ]:
### Implement the safety functionalities for the Robile by setting up
### a state machine and implementing all required states here

import rclpy
import smach

from std_msgs.msg import String
from sensor_msgs.msg import LaserScan
from geometry_msgs.msg import Twist
import yaml
import time

# Reference: https://wiki.ros.org/smach/Tutorials/Simple%20State%20Machine [but note that the wiki is just for ROS1]


class MonitorBatteryAndCollision(smach.State):
    """State to monitor the battery level and possible collisions"""

    def __init__(self, node):
        # TODO: define outcomes, class variables, and desired publisher/subscribers
        # YOUR CODE HERE
        raise NotImplementedError()

    def execute(self, userdata):
        # TODO: implement state execution logic and return outcome
        # YOUR CODE HERE
        raise NotImplementedError()


class RotateBase(smach.State):
    """State to rotate the Robile base"""

    def __init__(self, node):
        # TODO: define outcomes, class variables, and desired publisher/subscribers
        # YOUR CODE HERE
        raise NotImplementedError()

    def execute(self, userdata):
        # TODO: implement state execution logic and return outcome
        # YOUR CODE HERE
        raise NotImplementedError()


class StopMotion(smach.State):
    """State to stop the robot's motion"""

    def __init__(self, node):
        # TODO: define outcomes, class variables, and desired publisher/subscribers
        # YOUR CODE HERE
        raise NotImplementedError()

    def execute(self, userdata):
        # TODO: implement state execution logic and return outcome
        # YOUR CODE HERE
        raise NotImplementedError()


# TODO: define any additional states if necessary
### YOUR CODE HERE ###


def main(args=None):
    """Main function to initialise and execute the state machine"""

    # TODO: initialise a ROS2 node, set any threshold values, and define the state machine
    # YOUR CODE HERE
    raise NotImplementedError()


if __name__ == "__main__":
    main()

## Robot Safety Functionalities Using a Behaviour Tree [45 points]

The majority of implementations of behaviour trees in robotics are using `BehaviorTree.CPP` in cpp and [py_trees](https://py-trees.readthedocs.io/en/devel/) in Python. [py_trees_ros](https://py-trees-ros-tutorials.readthedocs.io/en/release-2.1.x/tutorials.html) is a wrapper for `py_trees` to integrate it with ROS, which we will use in this assignment.

Your task here is to implement the necessary behaviours and set up the behaviour tree to achieve the desired robot functionality.

Implement the required behaviours in the cell below. [35 points]

In [ ]:
### Implement the safety functionalities for the Robile by implementing all
### required behaviours here. Feel free to define additional behaviours if necessary

import rclpy
import py_trees as pt
import py_trees_ros as ptr
from geometry_msgs.msg import Twist
from std_msgs.msg import Float32
from sensor_msgs.msg import LaserScan
from rclpy.qos import QoSProfile, QoSReliabilityPolicy, QoSHistoryPolicy


class Rotate(pt.behaviour.Behaviour):
    """Rotates the robot about the z-axis"""

    def __init__(self, name="rotate platform", topic_name="/cmd_vel", ang_vel=1.0):
        super(Rotate, self).__init__(name)

        # TODO: initialise any necessary class variables
        # YOUR CODE HERE
        raise NotImplementedError()

    def setup(self, **kwargs):
        """Setting up things which generally might require time to prevent delay in the tree initialisation"""
        self.logger.info("[ROTATE] setting up rotate behaviour")

        try:
            self.node = kwargs["node"]
        except KeyError as e:
            error_message = "didn't find 'node' in setup's kwargs [{}][{}]".format(self.qualified_name)
            raise KeyError(error_message) from e

        # TODO: setup any necessary publishers or subscribers

        # YOUR CODE HERE
        raise NotImplementedError()

        return True

    def update(self):
        """Rotates the robot at the maximum allowed angular velocity.
        Note: The actual behaviour is implemented here.

        """
        self.logger.info("[ROTATE] update: updating rotate behaviour")
        self.logger.debug("%s.update()" % self.__class__.__name__)

        # TODO: implement the primary function of the behaviour and decide which status to return
        # based on the structure of your behaviour tree

        # Hint: to return a status, for example, SUCCESS, pt.common.Status.SUCCESS can be used

        # YOUR CODE HERE
        raise NotImplementedError()

    def terminate(self, new_status):
        """Trigerred once the execution of the behaviour finishes,
        i.e. when the status changes from RUNNING to SUCCESS or FAILURE
        """
        self.logger.info("[ROTATE] terminate: publishing zero angular velocity")

        # TODO: implement the termination of the behaviour, i.e. what should happen when the behaviour
        # finishes its execution

        # YOUR CODE HERE
        raise NotImplementedError()

        return super().terminate(new_status)


class StopMotion(pt.behaviour.Behaviour):
    """Stops the robot when it is controlled using a joystick or with a cmd_vel command"""

    # TODO: Implement a behaviour to stop the robot's motion

    # YOUR CODE HERE
    raise NotImplementedError()


class BatteryStatus2bb(ptr.subscribers.ToBlackboard):
    """Checks the battery status"""

    def __init__(self, battery_voltage_topic_name: str = "/battery_voltage", name: str = "Battery2BB", threshold: float = 30.0):
        super().__init__(
            name=name,
            topic_name=battery_voltage_topic_name,
            topic_type=Float32,
            blackboard_variables={"battery": "data"},
            initialise_variables={"battery": 100.0},
            clearing_policy=pt.common.ClearingPolicy.NEVER,  # to decide when data should be cleared/reset.
            qos_profile=ptr.utilities.qos_profile_unlatched(),
        )
        self.blackboard.register_key(key="battery_low_warning", access=pt.common.Access.WRITE)

        # YOUR CODE HERE
        raise NotImplementedError()

    def update(self):
        """Calls the parent to write the raw data to the blackboard and then check against the
        threshold to determine if a low warning flag should also be updated.
        """
        self.logger.info("[BATTERY] update: running battery_status2bb update")
        self.logger.debug("%s.update()" % self.__class__.__name__)

        """check battery voltage level stored in self.blackboard.battery. By comparing with 
        threshold value, update the value of self.blackboad.battery_low_warning
        """

        # TODO: based on the battery voltage level, update the value of self.blackboard.battery_low_warning
        # and return the status of the behaviour based on your logic of the behaviour tree

        # YOUR CODE HERE
        raise NotImplementedError()


class LaserScan2bb(ptr.subscribers.ToBlackboard):
    """Checks the laser scan measurements to avoid possible collisions."""

    def __init__(self, topic_name: str = "/scan", name: str = "Scan2BB", safe_range: float = 0.25):
        super().__init__(
            name=name,
            topic_name=topic_name,
            topic_type=LaserScan,
            blackboard_variables={"laser_scan": "ranges"},
            clearing_policy=pt.common.ClearingPolicy.NEVER,  # to decide when data should be cleared/reset.
            qos_profile=QoSProfile(reliability=QoSReliabilityPolicy.RMW_QOS_POLICY_RELIABILITY_BEST_EFFORT, history=QoSHistoryPolicy.RMW_QOS_POLICY_HISTORY_KEEP_LAST, depth=10),
        )

        # TODO: initialise class variables and blackboard variables
        # YOUR CODE HERE
        raise NotImplementedError()

    def update(self):
        # TODO: impletment the update function to check the laser scan data and update the blackboard variable
        # YOUR CODE HERE
        raise NotImplementedError()

Now, set up and initialise your behaviour tree in the cell below. [10 points]

In [ ]:
### Implement a behaviour tree using your previously implemented behaviours here

import py_trees as pt
import py_trees_ros as ptr
import operator

import py_trees.console as console
import rclpy
import sys


def create_root() -> pt.behaviour.Behaviour:
    """Structures a behaviour tree to monitor the battery status, and start
    to rotate if the battery is low and stop if it detects an obstacle in front of it.
    """

    # we define the root node
    root = pt.composites.Parallel(name="root", policy=pt.common.ParallelPolicy.SuccessOnAll(synchronise=False))

    ### we create a sequence node called "Topics2BB" and a selector node called "Priorities"
    topics2BB = pt.composites.Sequence("Topics2BB", memory=False)
    priorities = pt.composites.Selector("Priorities", memory=False)

    ### we create an "Idle" node, which is a running node to keep the robot idle
    idle = pt.behaviours.Running(name="Idle")

    """
    TODO:  The first and second level of the tree structure is defined above, but please
    define the rest of the tree structure.

    Class definitions for your behaviours are provided in behaviours.py; you also need to fill out
    the behaviour implementations!

    HINT: Some behaviours from pt.behaviours may be useful to use as well.
    """

    # YOUR CODE HERE
    raise NotImplementedError()

    # TODO: construct the behaviour tree structure using the nodes and behaviours defined above
    # HINT: for reference, the sample tree structure in the README.md file might be useful

    root.add_children([topics2BB, priorities])

    # YOUR CODE HERE
    raise NotImplementedError()

    return root


def main():
    """Initialises and executes the behaviour tree"""
    rclpy.init(args=None)

    root = create_root()
    tree = ptr.trees.BehaviourTree(root=root, unicode_tree_debug=True)

    try:
        tree.setup(timeout=30.0)
    except ptr.exceptions.TimedOutError as e:
        console.logerror(console.red + "failed to setup the tree, aborting [{}]".format(str(e)) + console.reset)
        tree.shutdown()
        rclpy.try_shutdown()
        sys.exit(1)
    except KeyboardInterrupt:
        # user-initiated shutdown
        console.logerror("tree setup interrupted")
        tree.shutdown()
        rclpy.try_shutdown()
        sys.exit(1)

    # frequency of ticks
    tree.tick_tock(period_ms=100)

    try:
        rclpy.spin(tree.node)
    except (KeyboardInterrupt, rclpy.executors.ExternalShutdownException):
        pass
    finally:
        tree.shutdown()
        rclpy.try_shutdown()


if __name__ == "__main__":
    main()

## Setting up Your System for Testing

1. (If not done already) Please set up Ubuntu 22.04, ROS2 humble, and the Robile simulation by following the [documentation](https://robile-amr.readthedocs.io/en/humble/).

2. Clone the `executive_smach` repository in the src folder of your workspace, and also install the `py-trees-ros` package:
```
cd ~/ros2_ws/src/  
git clone -b ros2 https://github.com/ros/executive_smach.git  
sudo apt-get install ros-humble-py-trees ros-humble-py-trees-ros ros-humble-py-trees-ros-interfaces xcb
```

From the workspace directory, build the workspace:
```
cd ~/ros2_ws/
colcon build --symlink-install
```

Now source the workspace setup file:
```
source install/setup.bash
```
3. Create a new ROS2 python package and integrate your implementation in it

## Testing Instructions

Use the following steps to test your implementation:
- Run the Robile in simulation
- After integrating your implementation in your local ROS workspace, launch your components to test the functionalities. **Note that you need to test the state machine and behaviour tree implementations independently to verify that the robot behaves exactly the same in both cases.**

**In your submission, please include screenshots to verify your implementation, and link them in the cell further below.**

As already mentioned before, as the battery percentage is not readily available in simulation, please publish the battery percentage values manually. For instance, if the topic `/battery_voltage` is used for monitoring the battery status, you should be able to publish a battery percentage value of your choice to verify your implementation, e.g.:
```  
ros2 topic  pub /battery_voltage std_msgs/msg/Float32 "data: 50.0"
```

Finally, behaviour tree visualization is not released on ROS2 humble yet, but the changes in the behaviour tree can be monitored by running the following command, which is helpful for debugging:
```
py-trees-tree-watcher
```

The following is a sample visualisation when the robot is about to collide:

![collision avoidance BT](figures/BT_watcher.png)

For getting a better intuition of behaviour trees, another sample visualisation of a similar task, but with a slightly different structure can be found below:

![collison and battery low](figures/collision_battery.png)

**Discuss any observations from your tests and include screenshots that verify your implementation in the cell below. [10 points]**

YOUR ANSWER HERE